In [1]:
# from importib import reload
%load_ext autoreload

In [2]:
import os
import sys
sys.path.append('..')

from scripts.utils.monitor import main
from scripts.utils.scraper import run_fetch_process
from scripts.data_utils.loaders import *
from scripts.data_utils.preprocess import *
from scripts.data_utils.extract import *
from scripts.data_utils.generator import *
from scripts.data_utils.tokenizer import *

%autoreload 2

c:\Users\Darkles\10academy\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Darkles\10academy\env\Lib\site-packages\amseg-2.3-py3.12.egg\amseg\amharicSegmenter.py:55: SyntaxWarning: invalid escape sequence '\s'
c:\Users\Darkles\10academy\env\Lib\site-packages\amseg-2.3-py3.12.egg\amseg\amharicSegmenter.py:79: SyntaxWarning: invalid escape sequence '\s'
c:\Users\Darkles\10academy\env\Lib\site-packages\amseg-2.3-py3.12.egg\amseg\amharicSegmenter.py:55: SyntaxWarning: invalid escape sequence '\s'
c:\Users\Darkles\10academy\env\Lib\site-packages\amseg-2.3-py3.12.egg\amseg\amharicSegmenter.py:79: SyntaxWarning: invalid escape sequence '\s'
c:\Users\Darkles\10academy\env\Lib\site-packages\amseg-2.3-py3.12.egg\amseg\amharicNormalizer.py:87: SyntaxWarning: invalid escape sequence '\s'
c:\Users\Darkles\10academ

In [18]:
CONFIG_PATH = os.path.join('..', 'resources', 'configs')
DATA_PATH = os.path.join('..', 'resources', 'data')

# Load channel usernames from a JSON file
channels_filepath = os.path.join(CONFIG_PATH, 'channels.json')
channels = load_json(channels_filepath)
CHANNELS = channels.get('CHANNELS', [])
CHANNELS

['ZemenExpress',
 'nevacomputer',
 'meneshayeofficial',
 'ethio_brand_collection',
 'Leyueqa',
 'sinayelj',
 'Shewabrand',
 'helloomarketethiopia',
 'modernshoppingcenter',
 'qnashcom',
 'Fashiontera',
 'kuruwear',
 'gebeyaadama',
 'MerttEka',
 'forfreemarket',
 'classybrands',
 'marakibrand',
 'aradabrand2',
 'marakisat2',
 'belaclassic',
 'AwasMart',
 'qnashcom']

In [5]:
raw_dir = os.path.join(DATA_PATH, 'raw')
processed_dir = os.path.join(DATA_PATH, 'processed')
labeled_dir = os.path.join(DATA_PATH, 'labeled')
os.makedirs(raw_dir, exist_ok=True)
os.makedirs(processed_dir, exist_ok=True)
os.makedirs(labeled_dir, exist_ok=True)

raw_file = os.path.join(raw_dir, 'all_channels_messages.csv')
processed_filename = 'preprocessed_messages'
labeled_filename = 'labeled_messages'

In [6]:
data = load_csv(raw_file)
data.info()

2025-01-20 11:46:16 - INFO - Loading data from file ...
2025-01-20 11:46:16 - INFO - Loading data from file ...
2025-01-20 11:46:16 - INFO - Loading data from file ...
2025-01-20 11:46:16 - INFO - Loading data from file ...
2025-01-20 11:46:16 - INFO - Loading data from file ...
2025-01-20 11:46:16 - INFO - Loading data from file ...
2025-01-20 11:46:16 - INFO - Loading data from file ...
2025-01-20 11:46:16 - INFO - Loading data from file ...
2025-01-20 11:46:16 - INFO - Loading data from file ...
2025-01-20 11:46:16 - INFO - Loading data from file ...
2025-01-20 11:46:16 - INFO - Loaded data from ..\resources\data\raw\all_channels_messages.csv, shape: (2465, 6)
2025-01-20 11:46:16 - INFO - Loaded data from ..\resources\data\raw\all_channels_messages.csv, shape: (2465, 6)
2025-01-20 11:46:16 - INFO - Loaded data from ..\resources\data\raw\all_channels_messages.csv, shape: (2465, 6)
2025-01-20 11:46:16 - INFO - Loaded data from ..\resources\data\raw\all_channels_messages.csv, shape: (2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2465 entries, 0 to 2464
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Channel Username  2465 non-null   object
 1   Message ID        2465 non-null   int64 
 2   Message           1315 non-null   object
 3   Date              2465 non-null   object
 4   Sender ID         2465 non-null   int64 
 5   Media             2144 non-null   object
dtypes: int64(2), object(4)
memory usage: 115.7+ KB


In [9]:
# Define regex patterns for entity labeling
PATTERNS = {
    "Product": re.compile(r"\b(ወርቅ|ሸማት|ኪሳራ|ሻርቤት}|በቃል|ፍሬ|ነገር|አካሉ|ቤት)\b"),
    "PRICE": re.compile(r"\b(በ|ዋጋ)\s?\d+\s?ብር\b"),
    "LOC": re.compile(r"\b(አዲስ\s?አበባ|ቦሌ|መቐለ|አዳማ|ሲዳማ|ሱቅ|ነዋሪ)\b"),
}

column = "Message"
labeled_data = generate_labels(preprossed_data, column, PATTERNS)
labeled_data

2025-01-20 11:46:18 - INFO - Starting to generate labels.
2025-01-20 11:46:18 - INFO - Starting to generate labels.
2025-01-20 11:46:18 - INFO - Starting to generate labels.
2025-01-20 11:46:18 - INFO - Starting to generate labels.
2025-01-20 11:46:18 - INFO - Starting to generate labels.
2025-01-20 11:46:18 - INFO - Starting to generate labels.
2025-01-20 11:46:18 - INFO - Starting to generate labels.
2025-01-20 11:46:18 - INFO - Starting to generate labels.
2025-01-20 11:46:18 - INFO - Starting to generate labels.
2025-01-20 11:46:18 - INFO - Starting to generate labels.
2025-01-20 11:46:18 - INFO - Starting to assign entity labels.
2025-01-20 11:46:18 - INFO - Starting to assign entity labels.
2025-01-20 11:46:18 - INFO - Starting to assign entity labels.
2025-01-20 11:46:18 - INFO - Starting to assign entity labels.
2025-01-20 11:46:18 - INFO - Starting to assign entity labels.
2025-01-20 11:46:18 - INFO - Starting to assign entity labels.
2025-01-20 11:46:18 - INFO - Starting to a

,Message,Tokens,Labels,Tokens_Labels
0,390 526 ስልክ 2519453526 ፋሽን ተራ አድራሻ አዲስ አበባ ጦር ...,"[390, 526, ስልክ, 2519453526, ፋሽን, ተራ, አድራሻ, አዲስ...","[O, O, O, O, O, O, O, B-LOC, I-LOC, O, O, O, O...","[(390, O), (526, O), (ስልክ, O), (2519453526, O)..."
1,43 250 526 ስልክ 2519453526 ፋሽን ተራ አድራሻ አዲስ አበባ ...,"[43, 250, 526, ስልክ, 2519453526, ፋሽን, ተራ, አድራሻ,...","[O, O, O, O, O, O, O, O, B-LOC, I-LOC, O, O, O...","[(43, O), (250, O), (526, O), (ስልክ, O), (25194..."
2,2 ፋሽን ተራ አድራሻ አዲስ አበባ ጦር ሀይሎች ድሪም ታወር 2ተኛ ፎቅ ቢ...,"[2, ፋሽን, ተራ, አድራሻ, አዲስ, አበባ, ጦር, ሀይሎች, ድሪም, ታወ...","[O, O, O, O, B-LOC, I-LOC, O, O, O, O, O, O, O...","[(2, O), (ፋሽን, O), (ተራ, O), (አድራሻ, O), (አዲስ, B..."
3,414243 350 526 ስልክ 2519453526 ፋሽን ተራ አድራሻ አዲስ ...,"[414243, 350, 526, ስልክ, 2519453526, ፋሽን, ተራ, አ...","[O, O, O, O, O, O, O, O, B-LOC, I-LOC, O, O, O...","[(414243, O), (350, O), (526, O), (ስልክ, O), (2..."
4,4243 340 526 ስልክ 2519453526 ፋሽን ተራ አድራሻ አዲስ አበ...,"[4243, 340, 526, ስልክ, 2519453526, ፋሽን, ተራ, አድራ...","[O, O, O, O, O, O, O, O, B-LOC, I-LOC, O, O, O...","[(4243, O), (340, O), (526, O), (ስልክ, O), (251..."
...,...,...,...,...
1310,3 የመፀዳጃ ቤትእቃዎች መደርደሪያ ጠንካራ ብረት ሼልፉ የላሜራ ብዙ ነገሮ...,"[3, የመፀዳጃ, ቤትእቃዎች, መደርደሪያ, ጠንካራ, ብረት, ሼልፉ, የላሜ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[(3, O), (የመፀዳጃ, O), (ቤትእቃዎች, O), (መደርደሪያ, O),..."
1311,3 የሴራሚክ መጥበሻ 24 26 28 ሶስቱን በአንድ ላይ ዋጋ 380 ብር አ...,"[3, የሴራሚክ, መጥበሻ, 24, 26, 28, ሶስቱን, በአንድ, ላይ, ዋ...","[O, O, O, O, O, O, O, O, O, B-PRICE, I-PRICE, ...","[(3, O), (የሴራሚክ, O), (መጥበሻ, O), (24, O), (26, ..."
1312,ለወይን እና ለተለያዩ የመጠጥ ማቅረቢያ እና መጠጫ ብርጭቆዎች 7 ውበትን ...,"[ለወይን, እና, ለተለያዩ, የመጠጥ, ማቅረቢያ, እና, መጠጫ, ብርጭቆዎች...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[(ለወይን, O), (እና, O), (ለተለያዩ, O), (የመጠጥ, O), (ማ..."
1313,ሳቺ የሊጥ ማቡኪያ እና መቀላቀያ 45 ሊጥ ማቡኪያ እንቁላል መምቻ ክሬም ...,"[ሳቺ, የሊጥ, ማቡኪያ, እና, መቀላቀያ, 45, ሊጥ, ማቡኪያ, እንቁላል...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[(ሳቺ, O), (የሊጥ, O), (ማቡኪያ, O), (እና, O), (መቀላቀያ..."


In [10]:
# Convert to CoNLL format
token_column = 'Tokens'
label_column = 'Labels'
tokens_labels_column = 'Tokens_Labels'

output_filename = os.path.join(labeled_dir, "Labeled_Messages_NER.conll")
output_filename_2 = os.path.join(labeled_dir, "Labeled_Messages_NER_2.conll")

save_conll_format(labeled_data, output_filename, column=tokens_labels_column)
save_conll_format(labeled_data, output_filename_2, columns=[token_column, label_column])

tokens_labels = labeled_data[tokens_labels_column]
tokens_labels

2025-01-20 11:47:05 - INFO - Saving labeled data to ..\resources\data\labeled\Labeled_Messages_NER.conll in CoNLL format.
2025-01-20 11:47:05 - INFO - Saving labeled data to ..\resources\data\labeled\Labeled_Messages_NER.conll in CoNLL format.
2025-01-20 11:47:05 - INFO - Saving labeled data to ..\resources\data\labeled\Labeled_Messages_NER.conll in CoNLL format.
2025-01-20 11:47:05 - INFO - Saving labeled data to ..\resources\data\labeled\Labeled_Messages_NER.conll in CoNLL format.
2025-01-20 11:47:05 - INFO - Saving labeled data to ..\resources\data\labeled\Labeled_Messages_NER.conll in CoNLL format.
2025-01-20 11:47:05 - INFO - Saving labeled data to ..\resources\data\labeled\Labeled_Messages_NER.conll in CoNLL format.
2025-01-20 11:47:05 - INFO - Saving labeled data to ..\resources\data\labeled\Labeled_Messages_NER.conll in CoNLL format.
2025-01-20 11:47:05 - INFO - Saving labeled data to ..\resources\data\labeled\Labeled_Messages_NER.conll in CoNLL format.
2025-01-20 11:47:05 - IN

0       [(390, O), (526, O), (ስልክ, O), (2519453526, O)...
1       [(43, O), (250, O), (526, O), (ስልክ, O), (25194...
2       [(2, O), (ፋሽን, O), (ተራ, O), (አድራሻ, O), (አዲስ, B...
3       [(414243, O), (350, O), (526, O), (ስልክ, O), (2...
4       [(4243, O), (340, O), (526, O), (ስልክ, O), (251...
                              ...                        
1310    [(3, O), (የመፀዳጃ, O), (ቤትእቃዎች, O), (መደርደሪያ, O),...
1311    [(3, O), (የሴራሚክ, O), (መጥበሻ, O), (24, O), (26, ...
1312    [(ለወይን, O), (እና, O), (ለተለያዩ, O), (የመጠጥ, O), (ማ...
1313    [(ሳቺ, O), (የሊጥ, O), (ማቡኪያ, O), (እና, O), (መቀላቀያ...
1314    [(የራስ, O), (ቅል, O), (እንዲሁም, O), (የፀጉር, O), (ማሻ...
Name: Tokens_Labels, Length: 1315, dtype: object

In [11]:
tokens, labels = separate_tokens_and_labels(tokens_labels)
len(tokens), len(labels)

2025-01-20 11:47:06 - INFO - Starting to separate tokens and labels from the data.
2025-01-20 11:47:06 - INFO - Starting to separate tokens and labels from the data.
2025-01-20 11:47:06 - INFO - Starting to separate tokens and labels from the data.
2025-01-20 11:47:06 - INFO - Starting to separate tokens and labels from the data.
2025-01-20 11:47:06 - INFO - Starting to separate tokens and labels from the data.
2025-01-20 11:47:06 - INFO - Starting to separate tokens and labels from the data.
2025-01-20 11:47:06 - INFO - Starting to separate tokens and labels from the data.
2025-01-20 11:47:06 - INFO - Starting to separate tokens and labels from the data.
2025-01-20 11:47:06 - INFO - Starting to separate tokens and labels from the data.
2025-01-20 11:47:06 - INFO - Starting to separate tokens and labels from the data.
2025-01-20 11:47:06 - WARNING - Empty item found in data. Skipping...
2025-01-20 11:47:06 - WARNING - Empty item found in data. Skipping...
2025-01-20 11:47:06 - WARNING 

(52537, 52537)

In [ ]:
# Tokenizer and Data Preparation
model_name = "xlm-roberta-base"
auto_tokenizer = initialize_auto_tokenizer(model_name)
auto_aligned_tokens, auto_aligned_labels = tokenize_and_align_labels(auto_tokenizer, tokens, labels)

In [13]:
# Display the first 20 results
for token, label in zip(auto_aligned_tokens[:20], auto_aligned_labels[:20]):
    print(f"{token:20} {label}")

▁390                 O
▁5                   O
26                   O
▁ስልክ                 O
▁25                  O
1945                 O
35                   O
26                   O
▁                    O
ፋ                    O
ሽን                   O
▁ተራ                  O
▁አድራሻ                O
▁አዲስ                 B-LOC
▁አበባ                 I-LOC
▁ጦር                  O
▁                    O
ሀይ                   O
ሎች                   O
▁ድ                   O


In [14]:
# Saving the final tokens and labels to a CoNLL and CSV/JSON file
save_labels(auto_aligned_tokens, auto_aligned_labels, f'{labeled_filename}_auto', labeled_dir, save_json=True)

2025-01-20 11:47:19 - INFO - Labels saved to CoNLL file: ..\resources\data\labeled\labeled_messages_auto.conll
2025-01-20 11:47:19 - INFO - Labels saved to CoNLL file: ..\resources\data\labeled\labeled_messages_auto.conll
2025-01-20 11:47:19 - INFO - Labels saved to CoNLL file: ..\resources\data\labeled\labeled_messages_auto.conll
2025-01-20 11:47:19 - INFO - Labels saved to CoNLL file: ..\resources\data\labeled\labeled_messages_auto.conll
2025-01-20 11:47:19 - INFO - Labels saved to CoNLL file: ..\resources\data\labeled\labeled_messages_auto.conll
2025-01-20 11:47:19 - INFO - Labels saved to CoNLL file: ..\resources\data\labeled\labeled_messages_auto.conll
2025-01-20 11:47:19 - INFO - Labels saved to CoNLL file: ..\resources\data\labeled\labeled_messages_auto.conll
2025-01-20 11:47:19 - INFO - Labels saved to CoNLL file: ..\resources\data\labeled\labeled_messages_auto.conll
2025-01-20 11:47:19 - INFO - Labels saved to CoNLL file: ..\resources\data\labeled\labeled_messages_auto.conll
2

In [15]:
# Initialize the Amharic segmenter
sent_punct, word_punct = [], []
segmenter = initialize_amharic_segmenter(sent_punct, word_punct)
new_tokens, new_labels = tokenize_and_align_labels(segmenter, tokens, labels)

2025-01-20 11:47:20 - INFO - Starting to initialize AmharicSegmenter
2025-01-20 11:47:20 - INFO - Starting to initialize AmharicSegmenter
2025-01-20 11:47:20 - INFO - Starting to initialize AmharicSegmenter
2025-01-20 11:47:20 - INFO - Starting to initialize AmharicSegmenter
2025-01-20 11:47:20 - INFO - Starting to initialize AmharicSegmenter
2025-01-20 11:47:20 - INFO - Starting to initialize AmharicSegmenter
2025-01-20 11:47:20 - INFO - Starting to initialize AmharicSegmenter
2025-01-20 11:47:20 - INFO - Starting to initialize AmharicSegmenter
2025-01-20 11:47:20 - INFO - Starting to initialize AmharicSegmenter
2025-01-20 11:47:20 - INFO - Starting to initialize AmharicSegmenter
2025-01-20 11:47:20 - INFO - Finished initializing AmharicSegmenter
2025-01-20 11:47:20 - INFO - Finished initializing AmharicSegmenter
2025-01-20 11:47:20 - INFO - Finished initializing AmharicSegmenter
2025-01-20 11:47:20 - INFO - Finished initializing AmharicSegmenter
2025-01-20 11:47:20 - INFO - Finished 

In [16]:
# Output the first 20 aligned tokens and labels
for token, label in zip(new_tokens[:20], new_labels[:20]):
    print(f"{token}: {label}")

390: O
526: O
ስልክ: O
2519453526: O
ፋሽን: O
ተራ: O
አድራሻ: O
አዲስ: B-LOC
አበባ: I-LOC
ጦር: O
ሀይሎች: O
ድሪም: O
ታወር: O
2ተኛ: O
ፎቅ: O
ቢሮ: O
ቁጥር: O
205: O
43: O
250: O


In [17]:
# Saving the final tokens and labels to a CoNLL and CSV/JSON file
save_labels(new_tokens, new_labels, f'{labeled_filename}_amseg', labeled_dir, save_json=True)

2025-01-20 11:47:21 - INFO - Labels saved to CoNLL file: ..\resources\data\labeled\labeled_messages_amseg.conll
2025-01-20 11:47:21 - INFO - Labels saved to CoNLL file: ..\resources\data\labeled\labeled_messages_amseg.conll
2025-01-20 11:47:21 - INFO - Labels saved to CoNLL file: ..\resources\data\labeled\labeled_messages_amseg.conll
2025-01-20 11:47:21 - INFO - Labels saved to CoNLL file: ..\resources\data\labeled\labeled_messages_amseg.conll
2025-01-20 11:47:21 - INFO - Labels saved to CoNLL file: ..\resources\data\labeled\labeled_messages_amseg.conll
2025-01-20 11:47:21 - INFO - Labels saved to CoNLL file: ..\resources\data\labeled\labeled_messages_amseg.conll
2025-01-20 11:47:21 - INFO - Labels saved to CoNLL file: ..\resources\data\labeled\labeled_messages_amseg.conll
2025-01-20 11:47:21 - INFO - Labels saved to CoNLL file: ..\resources\data\labeled\labeled_messages_amseg.conll
2025-01-20 11:47:21 - INFO - Labels saved to CoNLL file: ..\resources\data\labeled\labeled_messages_amse

In [ ]:
# Example Usage
tokens = ["አንድ", "እቃ", "ነኝ"]
labels = ["O", "B-Product", "I-Product"]
segmenter = AutoTokenizer.from_pretrained("xlm-roberta-base")

tokenized_tokens, aligned_labels = tokenize_and_align_labels(segmenter, tokens, labels)

print("Tokenized Tokens:", tokenized_tokens)
print("Aligned Labels:", aligned_labels)


In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

TRAIN_DATA_PATH = "resources/data/ner_train.conll"
VAL_DATA_PATH = "resources/data/ner_val.conll"

# Load the train and validation data
train_data = load_conll_data(TRAIN_DATA_PATH)
val_data = load_conll_data(VAL_DATA_PATH)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)


# Map function and format datasets
label_mapping = ClassLabel(names=["O", "B-Product", "I-Product", "B-LOC", "I-LOC", "B-PRICE", "I-PRICE"])

# Load Model
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=len(label_mapping.names))



train_dataset = train_dataset.map(lambda x: {"labels": [[label_mapping.str2int(tag) for tag in seq] for seq in x["labels"]]})
val_dataset = val_dataset.map(lambda x: {"labels": [[label_mapping.str2int(tag) for tag in seq] for seq in x["labels"]]})

train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
val_dataset = val_dataset.map(tokenize_and_align_labels, batched=True)




In [ ]:
# Training Arguments
training_args = TrainingArguments(
    output_dir="./models/ner_finetuned",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train Model
trainer.train()